## 1. Data Collection
#### 1.1 Get the list of books
The starting point of the Homework 3 is to collected the data on which we are going to work on.
What we want is to collect the url associated to each book in the list provided by this [link](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1). \
We save all the links in the list.txt file; each line corresponds to a book's url.

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import os
import re
import csv
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from field_functions import * #our script .py 
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [22]:
filename="page1.html"
prefix="https://www.goodreads.com"

With the following loop we extract the 100 links from each of the 300 pages:

In [29]:
for i in range(1,301):
    filename="page"+str(i)+".html"
    directory= "G:\ADM3\\" +str(i)+"\\"
    prefix="https://www.goodreads.com"

    f=open(directory+filename,encoding="utf-8")
    lines=f.readlines()
    
    links=[]
    urls=[]
    for line in lines:
        if 'class="bookTitle"' in line:
            links.append(line)
    for link in links:
        link=prefix+re.findall(r'(?<=href=")(.*?)(?=")',link)[0] 
        urls.append(link)
    
    for url in urls:
        with open(directory+"links.txt", "a") as text_file:
            text_file.write(url+"\n")

#### 1.2 Crawl books
After collecting the urls, we download the html corresponding to each of them. We save each page into an html file in order to make the page static and also avoid problem during the process of data colletion.\
With the following loop we download the 300 pages.

In [ ]:
for i in range(1,301):
    link="https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)
    cnt = requests.get(link)
    soup = BeautifulSoup(cnt.content, features="lxml")
    name="G:\ADM3\\"+str(i)+"\\"+"page"+str(i)+".html"
    print(name)
    f = open(name, "w",encoding="utf-8")
    f.write(soup.prettify())
    f.close()
    print("Pagina "+str(i)+" finita")

Then with this third loop we download the book pages. In the for loop we go thought the 300 pages and with the second one we get from each page the all the links (100).

In [ ]:
s=requests.Session()
for i in range(1,301):
    file= "G:\ADM3\\" +str(i)+"\\"+"links.txt"
    f=open(file,encoding="utf-8")
    lines=f.readlines()
    j=1
    for url in lines:
        cnt = requests.get(url)
        soup = BeautifulSoup(cnt.content, features="html.parser")
        name="G:\ADM3\\"+str(i)+"\\"+str(j)+".html"
        f = open(name, "w",encoding="utf-8")
        f.write(soup.prettify())
        f.close()
        j+=1
    print("Pagina "+str(i)+" finita")

Actually we had run this loop more than one time on the files with size of less than 100000 bytes. Most of them were downloaded correctly this time, but some, especially in the last pages (150-300), were actually "bad pages" on 
the website. \
In the next step, during parsing, we decided to ignore pages with size of less than 100000 bytes.

### 1.3 Parse downloaded pages
Once we have the html documents containing all the books in the 300 pages, we want extract specific information from each html. \
Here the list of what we are looking:
- Title 
- Series 
- Authors
- Ratings
- Number of givent ratings 
- Number of reviews 
- The entire plot 
- Number of pages 
- Published 
- Characters
- Setting
- Url

For this purpose we define a class, one for each point in the list, and in every class there are 3 methods:
* **name()** : it returns the name of each item (bookTitle,bookSeries,ecc...)
* **parse()** : using the method soup.find() it returns the information we are looking for that are contained in the html of the web page concerning the book
* **score()** : it computes the value of the score (we use this method in point 3 of the homework) 

We decide to build this structure in case one wants to implement new fields in the future.

After collecting all the information in which we are interested, now it's time to save them properly. \
We create a file 'article_index.tsv' for each book in which the first row is the **header** and from the second one there are the information in order splitted by a tab. In this way: \
bookTitle \t bookSeries \t bookAuthors \t ratingValue \t ratingCount \t reviewCount \t Plot \t NumberOfPages \t Publishing_Date \t Characters \t Setting \t Url \
The Hunger Games \t The Hunger Games \t   ...  \t District 12, Panem, Capitol, Panem, Panem (United States) \


If one information is missing, instead of the dots there is an empty space.

In [ ]:
functions=[bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberOfPages,Publishing_Date,\
           Characters,Setting,Url]

header = ""
for fun in functions[:-1]:
    header += fun.name()+"\t"
header +=functions[-1].name()
count = 1 #for the articles

for i in range(1,301):
    for j in range(1,101):
        name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(i)+"\\"+str(j)+".html"
        try:
            with open(name,"r",encoding="utf-8") as file:
                if os.path.getsize(name)<100000: #ignoring "bad pages"
                    continue
                soup=BeautifulSoup(file,features="html.parser")
                second_line = ""
                for fun in functions[:-1]:
                    try:
                        second_line += fun.parse(soup)+"\t"
                    except:
                        print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i)
                        second_line += "" + "\t"
                try:
                    second_line += functions[-1].parse(soup)  
                except:
                    print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i) 
                    second_line += "" + "\t"
            name="C:\\Users\\Stefania\\ADM_HW3\\articles\\articles_"+str(count)+".tsv"
            with open(name,"w",encoding="utf-8") as file:
                file.write(header)
                file.write("\n")
                file.write(second_line)
            count +=1
        except FileNotFoundError:
            continue

In the *best book ever list* there are also foreign books. Since for this analysis we are only interested in english books, here we check if the plot of each book is or not in english. If not, we remove these books from our list.

In [ ]:
count = []
for i in range(1,29226):
    with open('articles/articles_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            if custom_detect(row['Plot']) != 'en':
                count.append(i)
                print(i)

for i in count:
    os.remove('articles/articles_' + str(i) +'.tsv')

After removed the "bad plot" we reindex all the articles.

In [ ]:
counter = 1
for i in range(1, 29226):
    try:
        os.rename('articles/articles_' + str(i) +'.tsv', 'articles/article_' + str(counter) +'.tsv')
        counter += 1
    except FileNotFoundError:
        continue

## 2. Search Engine

In [7]:
from Search_engines import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter

#### 2.0 Data preprocessing
To see the code used to create the pkl files look at the "DataStructures" notebook.

In [ ]:
tokenizer = RegexpTokenizer(r'[a-z]+') 
stop_words = set(stopwords.words("english"))
stemmer= PorterStemmer()

In [ ]:
# - keys: index of each word from 0 to 55037
# - values: each book containing the unique word (index number)
with open('inverted_index_1.pkl', 'rb') as handle:
    inverted_index = pickle.load(handle) 

# - keys: all words in the all documents
# - values: index of each word from 0 to 55037
with open('vocabulary.pkl', 'rb') as handle:
    vocabulary = pickle.load(handle)

# - keys: index of each word from 0 to 55037
# - values: number of times a word appear in all books
with open('vocabulary2.pkl', 'rb') as handle:
    vocabulary2 = pickle.load(handle)

# - keys: index of each word from 0 to 55037
# - values: (book, TfIdf) for all the keys 
with open('tfIdf_index.pkl', 'rb') as handle:
    tfIdf_index = pickle.load(handle)

# - keys: book
# - values: (words, TfIdf) for all the words in the plot of the book
with open('BookTokens.pkl', 'rb') as handle:
    BookTokens = pickle.load(handle)

### 2.1 Conjunctive query
In this first search engine what we do is very simple: according to the query provided in input we return all the books that have in the plot that query term(s). The output format is: 
- bookTitle:
- Plot:
- Url:

In [ ]:
SearchEngine2_1()

### 2.2 Conjunctive query and ranking score
After this, we build the second search engine in which we also compute the cosine similarity in order to deteminate how similar are the query word(s) and the plot's body. Now we sort the result according to this values.

In [ ]:
SearchEngine2_2()

## 3. Define a new score!
Here we define a new metric on which we compute the score for each fildes. For further explaination please refer to the files  **field_functions.py** and **Search_engines.py**.

In [ ]:
fields =[bookTitle ,bookSeries ,bookAuthors ,ratingValue ,ratingCount ,reviewCount ,NumberOfPages ,Publishing_Date,Characters ,Setting ,Url]
SearchEngine3(fields)

## 4. Make a nice visualization!
For considering the first 10 BookSeries in order of appearance, we use the following for loop:

In [8]:
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns

top_10_series=[]
for i in range(1,26544):
    if len(top_10_series)==10:
            break
    with open('articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            bookseries=row["bookSeries"]
            if bookseries and not re.findall(r'(?<=#)[0-9]?[-–]{1}',bookseries):
                bookseries=bookseries.split("#")[0].strip()
                if bookseries not in top_10_series:
                    top_10_series.append(bookseries)
top_10_series          

Using a regex expression, we are going to exclude the entire book serie (for example The Hunger Games #1-3) and in the dictionary *series_dict* we save the useful information such as: book title, number of pages and publishing date.

In [ ]:
series_dict={}
for series in top_10_series:
    series_dict[series]=[]
for i in range(1,26544):
    with open('articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            bookseries=row["bookSeries"]
            if bookseries and not re.findall(r'(?<=#)[0-9]?[-–]{1}',bookseries):
                bookseries=bookseries.split("#")[0].strip()
                if bookseries in series_dict:
                       series_dict[bookseries].append((row["bookTitle"],row["NumberOfPages"],row["Publishing_Date"].split()[-1]))

Before the plot, we order the books in every series by date:

In [ ]:
for series in series_dict:
    series_dict[series].sort(key=lambda x:x[2])

Now we can build the 2D plot. The x-axis is the years since publication of the first book, the y-axis is the cumulative series page count.

In [ ]:
plt.figure(figsize=(18,10))
for bookseries in series_dict:
    book_list=series_dict[bookseries]
    x=[]
    y=[]
    y_cum=0
    for book in book_list:
        x.append(int(book[2]))
        y_cum+=int(book[1])
        y.append(y_cum)
    plt.plot(x, y, 'o', x, y, '-')
    plt.legend(['Book'], loc='best')
plt.show()

For a better visualization of the single book serie, we provide also a bar plot.

In [ ]:
fig, axs= plt.subplots(5, 2,figsize=(20,33),squeeze=False)
fig.subplots_adjust(hspace=0.5)
n=0 # counter used to assign plots' position 
for bookseries in series_dict:
    book_list=series_dict[bookseries]
    x=[]
    y=[]
    y_cum=0
    for book in book_list:
        x.append(str(book[2]))
        y_cum+=int(book[1])
        y.append(y_cum)
    pal = sns.color_palette("icefire",len(y))
    axs[int(n/2), n - int(n/2)*2].bar(x,y,color = pal);
    axs[int(n/2), n-int(n/2)*2].title.set_text(bookseries);
    axs[int(n/2), n-int(n/2)*2].set_xlabel('Dates');
    axs[int(n/2), n-int(n/2)*2].set_ylabel('Number of pages');
    n+=1

## 5. Algorithmic Question
You are given a string written in english capital letters, for example S="CADFECEILGJHABNOPSTIRYOEABILCNR." You are asked to find the maximum length of a subsequence of characters that
is in alphabetical order. Among all the possible such sequences, you are asked to find the one that is the longest.
#### Point 5.1
Write a recursive program that, given a string, computes the length of the subsequence of maximum length that is in alphabetical order. Try some examples. Are the examples of short strings correct?

In [9]:
def recursive_lcs(X,Y,m,n):
    if m == 0 or n == 0: #if the two strings have length =0, then the LCS is 0
        return 0
    #otherwise we call recursivly the function to generate all subsequences of the given strings and to find the longest common subsequence
    elif X[m-1] == Y[n-1]:
        return 1 + recursive_lcs(X,Y,m-1,n-1)
    else:
        return max(recursive_lcs(X,Y,m,n-1), recursive_lcs(X,Y,m-1,n))
    
def length_lcs(str):
    X = str.upper() #convert the given string into uppercase
    Y="ABCDEFGHIJKLMNOPQRSTUVXYZ"
    m = len(X)
    n = len(Y)
    lcs = recursive_lcs(X,Y,m,n)
    return (lcs)

Provide a string in input and the function will find the lcs:

In [10]:
s = str(input())
string = ''.join(filter(str.isalpha, s))
print("LCS: ", length_lcs(string))

Hello world!
LCS:  4


#### Point 5.2

Show that the running time of the algorithm is exponential.

A string whose length is n, has $2^n-1$ different possible subsequences, without considering the subsequence with length 0.
So the time complexity of the longest common subsequence with a recursive approach is $O(n 2^n)$. 
We need $O(n)$ time to check if a subsequence is common to both the strings. The algorithm will take $O(2^n)$ in worst case that happens when all characters of the two provived strings mismatch and so the length of LCS is 0.


#### Point 5.3
Write a program that computes the length of the subsequence of maximum length, using dynamic programming.

Finding the longest common subsequence between two strings is a maximization problem, and so we can apply the dynamic programming. According to this method we can use a support structure in which save the length of the subproblems.
This structure is the *look up table*. The element in position $[i][j]$ is the LCS of X(i) and Y(j). \
We iterate for the length of X, and for the length of Y; if the i-th element of X is 0, or the j-th element of Y is 0, the element in position $[i][j]$ is 0 as well. \
If the i-th element of X is equal to the j-th element of Y, the element in position $[i][j]=[i-1][j-1]+1$.\
The $+1$ at the end is because the last symbol is equal (thinking about the look up table, we are moving according to the diagonal and increase of 1).\
Otherwise, we chose the maximum value between the previus element in horizontal and the previus one in vertical.\
At the end we return the look_up_table[m][n] that contains the length of the longest common subsequence of $X[0,m-1]$ and $Y[0,n-1]$.

In [11]:
def memoization_lcs(X,Y):
    m = len(X)
    n = len(Y)
    look_up_table = [[None] * (n+1) for i in range(m+1)]
    for i in range(m + 1): 
        for j in range(n + 1):
            if i == 0 or j == 0: 
                look_up_table[i][j] = 0 
            elif X[i-1] == Y[j-1]: 
                look_up_table[i][j] = look_up_table[i-1][j-1] + 1 
            else: 
                look_up_table[i][j] = max(look_up_table[i-1][j], look_up_table[i][j-1])
    return look_up_table[m][n] 

def length_lcs(str):
    X = str.upper() #convert the given string into uppercase
    Y="ABCDEFGHIJKLMNOPQRSTUVXYZ"
    lcs = memoization_lcs(X,Y)
    return (lcs)

Provide a string in input and the function will find the lcs

In [12]:
s = str(input())
string = ''.join(filter(str.isalpha, s))
print("LCS: ", length_lcs(string))

Hello everyone!
LCS:  5


#### Point 5.4
What is its runtime complexity?

The run time complexity of this algorithm using the dynamic programmig is given by the dimension of the look up table because it also provide us the information about the space complexity of the subproblems. \
The dimension of the table is $n*m$ where the $m=len(X)$ and $n=len(Y)$. So the runtime complexity is $O(mn)$, since for computing each position in the table we need $O(1)$.